In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import ipywidgets as widgets
from ipywidgets import Dropdown
from IPython.display import clear_output, display

In [72]:
df=pd.read_csv('nhl_all_games_data.csv')
rink_img_path = "rink.png"
rink_img = mpimg.imread(rink_img_path)


In [7]:
xcol, ycol = "details_xCoord", "details_yCoord"
etype_col = "typeDescKey" if "typeDescKey" in df.columns else ("typeDesc" if "typeDesc" in df.columns else "type")


def draw_play_event(season, gameDate, event_id):
    clear_output(wait=True)

    df_game = df[(df['season'] == season) & (df['gameDate'] == gameDate)]
    if df_game.empty:
        print("No events for this game")
        return

    if event_id not in df_game['eventId'].unique():
        print(f"Event ID {event_id} not found in this game")
        return

    df_event = df_game[df_game['eventId'] == event_id]
    df_coords = df_event.dropna(subset=[xcol, ycol])

    fig, ax = plt.subplots(figsize=(10,6))
    ax.imshow(rink_img, extent=[-100,100,-42.5,42.5], zorder=0, aspect='auto')

    if not df_coords.empty:
        ax.scatter(df_coords[xcol], df_coords[ycol], c="blue", s=60, alpha=0.8, zorder=5)
        for _, row in df_coords.iterrows():
            label = f"{row.get(etype_col,'')} ({row.get('timeInPeriod','')})"
            ax.text(row[xcol]+2, row[ycol]+2, label, fontsize=8, color="black")

    ax.set_xlim(-100,100)
    ax.set_ylim(-42.5,42.5)
    ax.set_aspect("equal")
    ax.set_title(f"{season} | {gameDate} | Event {event_id}", fontsize=14)
    plt.show()

    display(df_event.reset_index(drop=True))


season_dd = widgets.Dropdown(options=sorted(df['season'].unique()), description="Season")
game_dd   = widgets.Dropdown(description="Game Date")
event_slider = widgets.SelectionSlider(description="Event ID", options=[0], value=0,
                                       continuous_update=False, layout=widgets.Layout(width="80%"))

def update_games(*args):
    season_val = season_dd.value
    games = sorted(df[df['season']==season_val]['gameDate'].unique())
    game_dd.options = games
    if games:
        game_dd.value = games[0]
season_dd.observe(update_games, names="value")

def update_events(*args):
    season_val = season_dd.value
    game_val = game_dd.value
    df_game = df[(df['season']==season_val) & (df['gameDate']==game_val)]
    event_ids = sorted(df_game['eventId'].unique())
    if len(event_ids) > 0:
        event_slider.options = event_ids
        event_slider.value = event_ids[0]
    else:
        event_slider.options = [0]
        event_slider.value = 0
game_dd.observe(update_events, names="value")

out = widgets.interactive_output(
    draw_play_event,
    {"season": season_dd, "gameDate": game_dd, "event_id": event_slider}
)


update_games()
update_events()


ui = widgets.VBox([season_dd, game_dd, event_slider, out])
display(ui)